In [3]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import requests
import time

# Google Sheets API 인증
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("/content/white-rune-235923-5a427af17fd9.json", scope)
client = gspread.authorize(creds)

# 시트 열기
spreadsheet = client.open("주식종목가져오기")

def fetch_stock_info(stock_code):
    padded_stock_code = str(stock_code).zfill(6)
    url = f"https://m.stock.naver.com/api/stock/{padded_stock_code}/integration"

    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        stock_info = {}

        total_infos = data.get('totalInfos', [])
        for info in total_infos:
            key = info.get('key')
            value = info.get('value')

            if key == "52주 최고":
                stock_info['high52Weeks'] = value
            elif key == "52주 최저":
                stock_info['low52Weeks'] = value
            elif key == "PER":
                stock_info['PER'] = value
            elif key == "EPS":
                stock_info['EPS'] = value
            elif key == "추정PER":
                stock_info['cnsPER'] = value
            elif key == "추정EPS":
                stock_info['cnsEPS'] = value
            elif key == "PBR":
                stock_info['PBR'] = value
            elif key == "BPS":
                stock_info['BPS'] = value
            elif key == "배당수익률":
                stock_info['dividendYieldRatio'] = value
            elif key == "주당배당금":
                stock_info['dividend'] = value

        return [
            stock_info.get('high52Weeks'), stock_info.get('low52Weeks'), stock_info.get('PER'),
            stock_info.get('EPS'), stock_info.get('cnsPER'), stock_info.get('cnsEPS'),
            stock_info.get('PBR'), stock_info.get('BPS'), stock_info.get('dividendYieldRatio'),
            stock_info.get('dividend')
        ]

    except requests.RequestException as e:
        print(f"Error fetching data for {stock_code}: {e}")
        return ["Error fetching data"]

def clear_range(sheet, range):
    cell_list = sheet.range(range)
    for cell in cell_list:
        cell.value = ''
    sheet.update_cells(cell_list)

def update_sheet(sheet_name):
    sheet = spreadsheet.worksheet(sheet_name)
    stock_codes = sheet.col_values(1)[1:]  # A2:A 컬럼 값 가져오기, 첫 번째 행 제외

    # 입력 범위 지우기
    clear_range(sheet, 'F2:O' + str(len(stock_codes) + 1))

    batch_data = []
    for i, stock_code in enumerate(stock_codes, start=2):
        stock_data = fetch_stock_info(stock_code)
        batch_data.append(stock_data)

    # 일괄 처리를 위해 데이터를 한 번에 업데이트
    batch_range = f'F2:O{len(stock_codes) + 1}'  # 데이터가 들어갈 범위 설정
    sheet.update(batch_range, batch_data)

# 코스피 시트 업데이트
update_sheet("코스피")

# 코스닥 시트 업데이트
update_sheet("코스닥")


<ipython-input-3-8b63434166e3>:82: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet.update(batch_range, batch_data)


Error fetching data for 388800: 409 Client Error: 409 for url: https://m.stock.naver.com/api/stock/388800/integration
